# 비화재보 분류 
---

## 1. 데이터 전처리     


In [1]:
import pandas as pd
import numpy as np 
import re

### 각 년도별 데이터 결합 

In [2]:
start_year= 2019
end_year =2021

df_all =pd.DataFrame()
#년도합치기
for year in range(start_year,end_year+1):
    globals()[f'df_{year}']=pd.DataFrame()
#Sheet 합치기
    for page in range(1,10):
        tmp = pd.read_excel(f"{year}년 전체 신고건수_관할 및 재난주소 추가.xls",sheet_name = f'Sheet {page}')['신고내용'].dropna(axis=0).T
        globals()[f'df_{year}']= pd.concat([globals()[f'df_{year}'],tmp], axis = 0)
    df_all= pd.concat([df_all,globals()[f'df_{year}']], axis = 0)
    

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

### 중복 문자열들 제거-set 활용

In [3]:
df_all.reset_index(inplace=True)
df_all.drop(columns=['index'],inplace=True)
df_all= pd.DataFrame(set(df_all[0]))

###  신고내용 중 숫자와 영어같은 비화재보 분류와 관련없는 단어들 제거 &전처리 

In [4]:
from konlpy.tag import Okt 
#속보기 단어인식을 못함 
okt = Okt()
print(okt.pos('속보기')) 
#속보기 ->속보로 단어 치환
print(okt.pos('경보기')) 
print(okt.pos('경보')) 
print(okt.pos('속보')) 

[('속', 'Modifier'), ('보기', 'Noun')]
[('경보기', 'Noun')]
[('경보', 'Noun')]
[('속보', 'Noun')]


In [5]:
df_all2 = df_all
for i in range(len(df_all2)):
    tmp = re.sub(r'[^가-힣+ ]',' ',df_all2.iloc[i,0]) # 한글말고 안나오게
    tmp = re.sub('속보기','속보',df_all2.iloc[i,0]) # 한글말고 안나오게
print(df_all2.head())
print(len(df_all2))

###  문장 -> 형태소 형태로 변환


In [ ]:
# 형태소 분석(Noun, Adjective) -명사와 형용사만
# 1자를 배제
# 불용어 제외
sents = []
for i in range(len(df_all2[0])):
    print("\r[tokenizing]  {}  /  {}   {}  %".format(i+1, len(df_all2[0]), round( (i / len(df_all2[0])) * 100 , 1 ) ), end = '    ', flush = False)
    pos_res = okt.pos(df_all2[0].iloc[i])
    
    
    STOP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    keywords = []
    for word, pos in pos_res:
        if ((pos == "Noun")|(pos == "Adjective"))&(len(word) >= 2)&(word not in STOP_WORDS):
            keywords.append(word)
    sents.append(' '.join(keywords))


[tokenizing]  28212  /  778936   3.6  %                            

KeyboardInterrupt: 

In [55]:
# tmp=pd.DataFrame(sents)
# tmp.to_csv("임시저장.csv",encoding= 'euc-kr')
# sents= pd.read_csv("임시저장.csv",encoding= 'euc-kr')
# sents =sents.values.tolist()

### 경보설비, 속보 설비   //      테스트, 오인주의, 훈련, 점검 문의"  키워드/갯수 확인

In [ ]:
df_all2[0].apply(lambda x: re.search("속보", x)is not None).sum()

0         None
1         None
2         None
3         None
4         None
          ... 
778931    None
778932    None
778933    None
778934    None
778935    None
Name: 0, Length: 778936, dtype: object

In [56]:
check = []
for i in range(len(sents)):
    for word in sents[i]:
        if '속보' in word: # 'a'가 포함된 모든 문자열 추출
            check.append(word)
print(len(check))     

5730


In [ ]:
df_all2[0].apply(lambda x: re.search("경보", x)is not None).sum()

In [57]:
check = []
for i in range(len(sents)):
    for word in sents[i]:
        if '경보' in word: # 'a'가 포함된 모든 문자열 추출
            check.append(word)
print(len(check))     

8477


In [85]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(df_all[0])
vect.vocabulary_

{'미군캠프워크헌병대남구청앞교통사고추가미군교통사고현장경찰출동중': 295276,
 '빨리오라고하고전화끊음추가보성황실동호추가동시통보': 348367,
 '구급상황센터이관수성구다리삐어거동불가병원안내': 103771,
 '삼주협화타운동호아버지거동불편하신데폐암환자열이오르고있다의식유': 363144,
 '수면제를알먹었다어머니의식유호흡유송현동층주택': 408960,
 '구급상황센터이관음주후수면제과다복용알정도상담': 131875,
 '도둑이들어와안내함': 242533,
 '금호지구사수동추가주취자지금여기가어딘지모른다위치': 164747,
 '무응답': 291855,
 '푸르지오호여성가슴답답해미치겠다': 673841,
 '개월애기방에갇혀있다지산동주택층': 14921,
 '구급상황센터이관애가살인데계속토하고설사하고청소년소아과안내본리동에있다': 116701,
 '중구동인동가주택층혈압이있으신데술을드셔서잘못일어나신다가슴답답하고춥다고하심대여성뇌경색병력': 601520,
 '구급상황센터이관약국안내요청': 122904,
 '신당동와룡공원동서서한아파트앞횡단보도추가피흘리고쓰러져': 427878,
 '층진압은다됐다담배꽁초화재자체진화완료': 639896,
 '통화전종료': 662751,
 '테크노공원로길불났다테크노락볼링장건물보물섬노래주점층에있는데연기가난다타는냄새도난다추가담배꽁초자체진화현풍제외귀소': 660990,
 '구급상황센터이관성인남성음주후다리가뭉쳐쥐나듯이상담': 100022,
 '남대구전화국네거리교통사고관련문의사고자아내로남편상태문의미국인추정상태알아보고전화연락하겠다고안내후재차통화시도했으나착신정지되어있는상태': 172695,
 '상수도터져인도블록위로물이흐린다서구통학로길': 365844,
 '모친복통허리통증심해거동불가백년가약호': 286981,
 '공경로길에사람이쓰러져주택앞추가움직임있다함': 43906,
 '칠곡지구동화골든빌동호애기가문에손이찍혔는데멍이들었다병원이송요청': 644886,
 '삼덕동인데공사장인데드럼통내에불꽃이조금보인다동성로길신자대기중': 360739,
 '동신교회전화번호좀가르쳐달라로연

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer().fit( 데이터 )
result =tfidf.fit_transform(데이터).toarray()
print(result)